# TUGAS 1 CRAWLING DATA

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
!pip install requests
!pip install html5lib
!pip install bs4

  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=07b79a15aa0bb5bffc31d69c385be5ce9afa11666032493da105a55234e1cd9b
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


In [ ]:
# constant
LAST_INDEX = -1
FIRST_INDEX = 1
INCREMENT_BY_ONE = 1

In [ ]:
# import library
from bs4 import BeautifulSoup
import requests

def request_url(url):
    return requests.get(url)

def request_header_url(header, url_website):
    headers = {'User-Agent': header}
    return requests.get(url=url_website, headers=headers)

def parse_website(request):
    """Use html5lib library to parse"""
    return BeautifulSoup(request.content, 'html5lib') # If this line causes an error, run 'pip install html5lib' or install html5lib

def prettify_web_structure(parsed_page):
    parsed_page.prettify()

def get_content_table(web_element, tag, attributes):
    return web_element.find(tag, attrs = attributes)

In [ ]:
# max_pages = 172 # total halaman di website PTA, terbaru februari 2023

r = request_header_url("Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:15.0) Gecko/20100101 Firefox/15.0.1", "https://pta.trunojoyo.ac.id/c_search/byprod/10")
soup = parse_website(r)
prettify_web_structure(soup)
table = get_content_table(soup, "div", {"id":"wrapper"})

pagination = table.findAll("a", attrs = {"class":"pag_button"})
total_pages = int(pagination[LAST_INDEX]["href"].split("/")[LAST_INDEX])

papers = []
for pages in range(FIRST_INDEX, total_pages + INCREMENT_BY_ONE):

    url_link = "https://pta.trunojoyo.ac.id/c_search/byprod/10"
    id_prodi = url_link.split("/")[LAST_INDEX]
    nama_prodi = ""
    if (id_prodi == "10"):
        nama_prodi = "Teknik Informatika"

    r_pages = request_header_url("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36", f"{url_link}/{pages}")
    soup_pages = parse_website(r_pages)
    prettify_web_structure(soup_pages)
    table_pages = get_content_table(soup_pages, "div", {"id":"wrapper"})

    for article_row in table_pages.findAll("a", attrs = {"class":"gray button"}):
        r_article_row = request_header_url("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36", article_row["href"])
        soup_article_row = parse_website(r_article_row)
        prettify_web_structure(soup_article_row)
        table_article_row = get_content_table(soup_article_row, "li", {"data-id":"id-1", "data-cat":"#luxury"})
        article_row_title = table_article_row.find('a', attrs = {"class":"title", "href":"#"}).string
        article_row_penulis = ""
        article_row_pembimbing1 = ""
        article_row_pembimbing2 = ""
        article_row_abstract = table_article_row.find('p', attrs = {"align":"justify"}).string
        spans = table_article_row.find_all('span')
        for span in spans:
          span_text = span.get_text(strip=True)
          if "Penulis :" in span_text:
            article_row_penulis = span_text.replace("Penulis :", "").strip()
          elif "Dosen Pembimbing I :" in span_text:
            article_row_pembimbing1 = span_text.replace("Dosen Pembimbing I :", "").strip()
          elif "Dosen Pembimbing II :" in span_text:
            article_row_pembimbing2 = span_text.replace("Dosen Pembimbing II :", "").strip()

        papers.append([
                article_row_title,
                article_row_penulis,
                article_row_pembimbing1,
                article_row_pembimbing2,
                article_row_abstract
        ])

In [ ]:
df = pd.DataFrame(papers, columns=["Judul", "Penulis", "pembimbing I", "pembimbing II", "Abstrak"])
df

,Judul,Penulis,pembimbing I,pembimbing II,Abstrak
0,PERANCANGAN DAN IMPLEMENTASI SISTEM DATABASE \...,A.Ubaidillah S.Kom,Budi Setyono M.T,Hermawan S.T,Sistem informasi akademik (SIAKAD) merupaka...
1,APLIKASI KONTROL DAN MONITORING JARINGAN KOMPU...,"M. Basith Ardianto,","Drs. Budi Soesilo, MT","Koko Joni, ST",Berjalannya koneksi jaringan komputer dengan l...
2,RANCANG BANGUN APLIKASI PROXY SERVER UNTUK\nEN...,"Akhmad Suyandi, S.Kom","Drs. Budi Soesilo, M.T","Hermawan, ST, MT",Web server adalah sebuah perangkat lunak serve...
3,SISTEM PENDUKUNG KEPUTUSAN OPTIMASI PENJADWALA...,Heri Supriyanto,"Mulaab, S.Si., M.Kom","Firli Irhamni, ST., M.Kom",Penjadwalan kuliah di Perguruan Tinggi me...
4,SISTEM AUGMENTED REALITY ANIMASI BENDA BERGERA...,Septian Rahman Hakim,"Arik Kurniawati, S.Kom., M.T.","Haryanto, S.T., M.T.",Seiring perkembangan teknologi yang ada diduni...
...,...,...,...,...,...
853,PENERAPAN ALGORITMA LONG-SHORT TERM MEMORY UNT...,Rachmad Agung Pambudi,"Eka Mala Sari Rochman, S.Kom., M.Kom","Sri Herawati, S.Kom., M.Kom",Investasi saham selama ini memiliki resiko ker...
854,SISTEM PENCARIAN TEKS AL-QURAN TERJEMAHAN BERB...,Nadila Hidayanti,"Achmad Jauhari, S.T., M.Kom","Ika Oktavia Suzanti, S.Kom., M.Cs",Information Retrieval (IR) merupakan pengambil...
855,KLASIFIKASI KOMPLEKSITAS VISUAL CITRA SAMPAH M...,Afni Sakinah,"Dr. Indah Agustien Siradjuddin, S.Kom., M.Kom.","Moch. Kautsar Sophan, S.Kom., M.MT.",Klasifikasi citra merupakan proses pengelompok...
856,IDENTIFIKASI BINER ATRIBUT PEJALAN KAKI MENGGU...,Friska Fatmawatiningrum,"Dr. Indah Agustien Siradjuddin, S.Kom., M.Kom.","Prof. Dr. Arief Muntasa, S.Si., M.MT.",Identifikasi atribut pejalan kaki merupakan sa...


In [ ]:
import os
df.to_csv('/content/drive/MyDrive/ppw/PTA.csv')

# TUGAS 2

**NORMALISASI TEKS**
1. Punctuation process menghilangkan tanda baca dan simbol

2. Stopword

3. Tokenisasi

4. Stemming

5. Feature extraction dan membentuk VSM dalam term frequency, logarithm freqency, one-hot encoding,  TF-IDF

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
#from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

import warnings
import pandas as pd
import numpy as np
import nltk
import re
import csv

nltk.download('stopwords')
nltk.download('punkt')
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Load Data

In [ ]:
data_x = pd.read_csv('/content/drive/MyDrive/ppw/PTA.csv')
data_x

,Unnamed: 0,Judul,Penulis,pembimbing I,pembimbing II,Abstrak
0,0,PERANCANGAN DAN IMPLEMENTASI SISTEM DATABASE \...,A.Ubaidillah S.Kom,Budi Setyono M.T,Hermawan S.T,Sistem informasi akademik (SIAKAD) merupaka...
1,1,APLIKASI KONTROL DAN MONITORING JARINGAN KOMPU...,"M. Basith Ardianto,","Drs. Budi Soesilo, MT","Koko Joni, ST",Berjalannya koneksi jaringan komputer dengan l...
2,2,RANCANG BANGUN APLIKASI PROXY SERVER UNTUK\nEN...,"Akhmad Suyandi, S.Kom","Drs. Budi Soesilo, M.T","Hermawan, ST, MT",Web server adalah sebuah perangkat lunak serve...
3,3,SISTEM PENDUKUNG KEPUTUSAN OPTIMASI PENJADWALA...,Heri Supriyanto,"Mulaab, S.Si., M.Kom","Firli Irhamni, ST., M.Kom",Penjadwalan kuliah di Perguruan Tinggi me...
4,4,SISTEM AUGMENTED REALITY ANIMASI BENDA BERGERA...,Septian Rahman Hakim,"Arik Kurniawati, S.Kom., M.T.","Haryanto, S.T., M.T.",Seiring perkembangan teknologi yang ada diduni...
...,...,...,...,...,...,...
853,853,PENERAPAN ALGORITMA LONG-SHORT TERM MEMORY UNT...,Rachmad Agung Pambudi,"Eka Mala Sari Rochman, S.Kom., M.Kom","Sri Herawati, S.Kom., M.Kom",Investasi saham selama ini memiliki resiko ker...
854,854,SISTEM PENCARIAN TEKS AL-QURAN TERJEMAHAN BERB...,Nadila Hidayanti,"Achmad Jauhari, S.T., M.Kom","Ika Oktavia Suzanti, S.Kom., M.Cs",Information Retrieval (IR) merupakan pengambil...
855,855,KLASIFIKASI KOMPLEKSITAS VISUAL CITRA SAMPAH M...,Afni Sakinah,"Dr. Indah Agustien Siradjuddin, S.Kom., M.Kom.","Moch. Kautsar Sophan, S.Kom., M.MT.",Klasifikasi citra merupakan proses pengelompok...
856,856,IDENTIFIKASI BINER ATRIBUT PEJALAN KAKI MENGGU...,Friska Fatmawatiningrum,"Dr. Indah Agustien Siradjuddin, S.Kom., M.Kom.","Prof. Dr. Arief Muntasa, S.Si., M.MT.",Identifikasi atribut pejalan kaki merupakan sa...


In [ ]:
data_x.isnull().sum()

Unnamed: 0        0
Judul             6
Penulis          10
pembimbing I     10
pembimbing II    11
Abstrak          30
dtype: int64

In [ ]:
data_x = data_x.dropna()
data_x.isnull().sum()

Unnamed: 0       0
Judul            0
Penulis          0
pembimbing I     0
pembimbing II    0
Abstrak          0
dtype: int64

# Punctuation Process

Menghapus tanda baca dan simbol.

In [ ]:
import re, string

In [ ]:
def cleaning(text):
    # HTML Tag Removal
    text = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});').sub('', str(text))

    # Case folding
    text = text.lower()

    # Trim text
    text = text.strip()

    # Remove punctuations, karakter spesial, and spasi ganda
    text = re.compile('<.*?>').sub('', text)
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)
    text = re.sub('\s+', ' ', text)

    # Number removal
    text = re.sub(r'\[[0-9]*\]', ' ', text)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d', ' ', text)
    text = re.sub(r'\s+', ' ', text)

    # Mengubah text 'nan' dengan whitespace agar nantinya dapat dihapus
    text = re.sub('nan', '', text)

    return text

data_x['Cleaning'] = data_x['Abstrak'].apply(cleaning)
data_x['Cleaning']

0      sistem informasi akademik siakad merupakan sis...
1      berjalannya koneksi jaringan komputer dengan l...
2      web server adalah sebuah perangkat lunak serve...
3      penjadwalan kuliah di perguruan tinggi merupak...
4      seiring perkembangan teknologi yang ada diduni...
                             ...                        
853    investasi saham selama ini memiliki resiko ker...
854    information retrieval ir merupakan pengambilan...
855    klasifikasi citra merupakan proses pengelompok...
856    identifikasi atribut pejalan kaki merupakan sa...
857    topik deteksi objek telah menarik perhatian ya...
Name: Cleaning, Length: 827, dtype: object

# Tokenisasi

In [ ]:
import nltk
nltk.download('popular')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

In [ ]:
def tokenizer(text):
  text = text.lower()
  return word_tokenize(text)

#data_x['Abstrak_Tokens'] = data_x['Abstrak'].apply(lambda x: word_tokenize(x))
#data_x[["Abstrak", "Abstrak_Tokens"]]

data_x['Tokenizing'] = data_x['Cleaning'].apply(tokenizer)
data_x['Tokenizing']

0      [sistem, informasi, akademik, siakad, merupaka...
1      [berjalannya, koneksi, jaringan, komputer, den...
2      [web, server, adalah, sebuah, perangkat, lunak...
3      [penjadwalan, kuliah, di, perguruan, tinggi, m...
4      [seiring, perkembangan, teknologi, yang, ada, ...
                             ...                        
853    [investasi, saham, selama, ini, memiliki, resi...
854    [information, retrieval, ir, merupakan, pengam...
855    [klasifikasi, citra, merupakan, proses, pengel...
856    [identifikasi, atribut, pejalan, kaki, merupak...
857    [topik, deteksi, objek, telah, menarik, perhat...
Name: Tokenizing, Length: 827, dtype: object

# Stopwords

In [ ]:
stop_words = set(stopwords.words('indonesian'))
print(stop_words)

{'asalkan', 'tinggi', 'mengetahui', 'nyaris', 'menuturkan', 'pentingnya', 'memberi', 'diperbuat', 'jikalau', 'awalnya', 'berturut', 'ditambahkan', 'selama-lamanya', 'disini', 'sebabnya', 'kepada', 'per', 'setiba', 'mempunyai', 'seperti', 'terdahulu', 'dikatakannya', 'sebagai', 'semuanya', 'sesuatu', 'semampu', 'belumlah', 'bolehkah', 'belakang', 'entah', 'pihaknya', 'belum', 'untuk', 'berlangsung', 'siapakah', 'ditanya', 'mulanya', 'bakal', 'kami', 'sekadar', 'mendatangkan', 'dipunyai', 'rupanya', 'sebegini', 'sebisanya', 'diucapkan', 'kalaupun', 'bahwa', 'mengenai', 'bertanya-tanya', 'mampukah', 'sebenarnya', 'sajalah', 'masalah', 'teringat', 'jadinya', 'padahal', 'sudah', 'pasti', 'yaitu', 'disebut', 'saja', 'berarti', 'diri', 'mengingatkan', 'mengungkapkan', 'menyatakan', 'diibaratkan', 'disampaikan', 'bersama-sama', 'entahlah', 'benarlah', 'tanyakan', 'ibu', 'ditandaskan', 'guna', 'antar', 'empat', 'kiranya', 'balik', 'rasanya', 'boleh', 'memperkirakan', 'kecil', 'akhiri', 'berujar

In [ ]:
corpus = stopwords.words('indonesian')

def stopwordText(words):
 return [word for word in words if word not in corpus]

data_x['Stopword Removal'] = data_x['Tokenizing'].apply(stopwordText)

# Gabungkan kembali token menjadi kalimat utuh
data_x['Full Text'] = data_x['Stopword Removal'].apply(lambda x: ' '.join(x))
data_x['Full Text']

0      sistem informasi akademik siakad sistem inform...
1      berjalannya koneksi jaringan komputer lancar g...
2      web server perangkat lunak server berfungsi me...
3      penjadwalan kuliah perguruan kompleks permasal...
4      seiring perkembangan teknologi didunia muncul ...
                             ...                        
853    investasi saham memiliki resiko kerugian perge...
854    information retrieval ir pengambilan informasi...
855    klasifikasi citra proses pengelompokan piksel ...
856    identifikasi atribut pejalan kaki salah peneli...
857    topik deteksi objek menarik perhatian perkemba...
Name: Full Text, Length: 827, dtype: object

# Stemming

In [ ]:
!pip install Sastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.4 MB/s eta 0:00:00


In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from tqdm.auto import tqdm
tqdm.pandas()

factory = StemmerFactory()
stemmer = factory.create_stemmer()

data_x['Stemming'] = data_x['Stopword Removal'].progress_apply(lambda x: stemmer.stem(' '.join(x)).split(' '))
#data['Abstrak_Tokens'] = data['Abstrak_Tokens'].progress_apply(lambda x: stemmer.stem(' '.join(x)).split(' '))
data_x['Stemming']

  0%|          | 0/827 [00:00<?, ?it/s]

0      [sistem, informasi, akademik, siakad, sistem, ...
1      [jalan, koneksi, jaring, komputer, lancar, gan...
2      [web, server, perangkat, lunak, server, fungsi...
3      [jadwal, kuliah, guru, kompleks, masalah, vari...
4      [iring, kembang, teknologi, dunia, muncul, tek...
                             ...                        
853    [investasi, saham, milik, resiko, rugi, gera, ...
854    [information, retrieval, ir, ambil, informasi,...
855    [klasifikasi, citra, proses, kelompok, piksel,...
856    [identifikasi, atribut, pejal, kaki, salah, te...
857    [topik, deteksi, objek, tarik, perhati, kemban...
Name: Stemming, Length: 827, dtype: object

# Feature Extraction

# One Hot Encoding

In [ ]:
def pandasOneHotEncoder(dokumens):
  encoder  = pd.get_dummies(dokumens.apply(pd.Series).stack()).sum(level=0)
  data_x = pd.concat([dokumens, encoder], axis=1)

  return data_x

oneHotEncoder = pandasOneHotEncoder(data_x['Stopword Removal'])
oneHotEncoder

,Stopword Removal,a,aalysis,aam,abad,abadi,ability,abjad,absensi,absolut,...,zcz,zf,zona,zone,zoning,zoom,zucara,zungu,µ,¼
0,"[sistem, informasi, akademik, siakad, sistem, ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"[berjalannya, koneksi, jaringan, komputer, lan...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"[web, server, perangkat, lunak, server, berfun...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"[penjadwalan, kuliah, perguruan, kompleks, per...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"[seiring, perkembangan, teknologi, didunia, mu...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
853,"[investasi, saham, memiliki, resiko, kerugian,...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
854,"[information, retrieval, ir, pengambilan, info...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
855,"[klasifikasi, citra, proses, pengelompokan, pi...",2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
856,"[identifikasi, atribut, pejalan, kaki, salah, ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
oneHotEncoder.to_csv('OneHotEncoder.csv', index=False)

# TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
def tfidf(dokumen):
  vectorizer = TfidfVectorizer()
  x = vectorizer.fit_transform(dokumen).toarray()
  terms = vectorizer.get_feature_names_out()

  tf_idf = pd.DataFrame(x, columns=terms)
  tf_idf.insert(0, 'Dokumen', dokumen)

  return (vectorizer, tf_idf)

tfidf_vectorizer, tf_idf = tfidf(data_x['Full Text'])
tf_idf

,Dokumen,aalysis,aam,abad,abadi,ability,abjad,absensi,absolut,absolute,...,zara,zat,zcz,zf,zona,zone,zoning,zoom,zucara,zungu
0,sistem informasi akademik siakad sistem inform...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,berjalannya koneksi jaringan komputer lancar g...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,web server perangkat lunak server berfungsi me...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,penjadwalan kuliah perguruan kompleks permasal...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,seiring perkembangan teknologi didunia muncul ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
822,tim futsal pelatih seleksi pemain pemain terba...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
823,kurangnya pemahaman gejala penyakit saluran pe...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
824,data set hilang utama studi bersifat substansi...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
825,proses seleksi penerimaan tenaga kerja faktor ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
tf_idf.to_csv('TF-IDF.csv', index=False)

# Term Frekuensi

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
def term_freq(dokumens):
  # Buat objek CountVectorizer
  vectorizer = CountVectorizer()
  tf_matrix = vectorizer.fit_transform(dokumens).toarray()
  terms = vectorizer.get_feature_names_out()

  tf = pd.DataFrame(tf_matrix, columns=terms)
  tf.insert(0, 'Dokumen', dokumens)

  return (vectorizer, tf, tf_matrix, terms)

tf_vectorizer, tf, tf_matrix, tf_terms = term_freq(data_x['Full Text'])
tf

,Dokumen,aalysis,aam,abad,abadi,ability,abjad,absensi,absolut,absolute,...,zara,zat,zcz,zf,zona,zone,zoning,zoom,zucara,zungu
0,sistem informasi akademik siakad sistem inform...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,berjalannya koneksi jaringan komputer lancar g...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,web server perangkat lunak server berfungsi me...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,penjadwalan kuliah perguruan kompleks permasal...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,seiring perkembangan teknologi didunia muncul ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
822,tim futsal pelatih seleksi pemain pemain terba...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
823,kurangnya pemahaman gejala penyakit saluran pe...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
824,data set hilang utama studi bersifat substansi...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
825,proses seleksi penerimaan tenaga kerja faktor ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
tf.to_csv('Term Frequensi.csv', index=False)

# Logarithm Frekuensi

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def log_freq(dokumens):
  return np.log10(dokumens + 1)

data_log_freq = pd.DataFrame(tf_matrix, columns=tf_terms).apply(log_freq)
data_log_freq.insert(0, 'Dokumen', data_x['Full Text'])
data_log_freq

,Dokumen,aalysis,aam,abad,abadi,ability,abjad,absensi,absolut,absolute,...,zara,zat,zcz,zf,zona,zone,zoning,zoom,zucara,zungu
0,sistem informasi akademik siakad sistem inform...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,berjalannya koneksi jaringan komputer lancar g...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,web server perangkat lunak server berfungsi me...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,penjadwalan kuliah perguruan kompleks permasal...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,seiring perkembangan teknologi didunia muncul ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
822,tim futsal pelatih seleksi pemain pemain terba...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
823,kurangnya pemahaman gejala penyakit saluran pe...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
824,data set hilang utama studi bersifat substansi...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
825,proses seleksi penerimaan tenaga kerja faktor ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
data_log_freq.to_csv('Logarithm Frequensi.csv', index=False)